### Connect 4 Python Implementation

In [1]:
# Import the required packages
import numpy as np
import random

First, we will create the basic functionalities of the game Connect4. For this, we will use a classs for the game configurations, and other for the two players (one AI and one human). The game uses a command line interface, where the user can enter the column number in which he/she wants to drop the piece into. Then, as this progresses, if there is a case where four pieces of the same symbol are connected, the player who does so wins.

The class for the AI and human player.

In [2]:
# Size of the grids
rows = 6
cols = 7

class Manual(object):
    """
      We have two classes for Player. This is the manual class.
      If a human chooses to play against an AI or against another human, we take the instance of this class.
      * type: Mantual Or Computer
      * identifier: Name or any symbol that is unique to the player
      * Symbol: Symbol that is shown in the user interface.
      Methods:
      * move(): Since this is manual, gives the user a prompt to make a decision about the move.
    """

    type = None
    identifier = None
    symbol = None
    def __init__(self, identifier, symbol):
        self.type = "Manual"
        self.identifier = identifier
        self.symbol = symbol
    
    # State is the current state of the board. And, based on that the decision is made.
    def move(self, state):
        print("This it the turn for {0}.  The symbol for {0} if {1}".format(self.identifier, self.symbol))
        
        column = None
        while column == None:
            try:
              # Ask the user to input their choice
                next_piece = int(input("Where would you like the next piece to fall.")) - 1
            except ValueError:
                next_piece = None
            # If the choice is between the number of columns in the grid, choice is valid
            if 0 <= next_piece and next_piece <= rows:
                column = next_piece
            else:

                print("Oops! Please choose a valid column")
        return column


class Computer(Manual):
  """ 
    There are few more features we add to the Computer class that we inherit.
    The computer uses Minimax algorithm to make the decision.
  """
  def __init__(self, identifier, symbol):
    self.type = "Computer"
    self.identifier = identifier
    self.symbol = symbol
  # The best predicted move will be the next move of the computer.
  def move(self, state):
    print("This it the turn for {0}.  The symbol for {0} if {1}".format(self.identifier, self.symbol))
    next_state = Minimax(state)
    # The depth is at 5, but it can be changed to achieve varying levels of difficulty.
    # The mode the depth, the higher the probability of winning, but slower the algorith.
    # 5 seems to be in the right spot.
    best_move, value = next_state.nextMoveAI(5, state, self.symbol)
    # Return the best move for the computer
    return best_move

Now, we have the class for the connect4 game iteself, that consists of all the mechanisms of the game including the win condition, turn rotations, and much more.

In [10]:
class ConnectFour(object):
  """
    Creates the Connect 4 game with:
    * Board state
    * Logic to keep track of turns
    * Checking if the four connections are made
    * Creating a user-friendly command interface to play the game
  """
  players = [None, None] # Connect 4 has two players
  ended, rotation, state, winner, turns = None, None, None, None, None
  symbols = [None, None]
  ROWS = rows
  COLS = cols

  def __init__(self):
    self.ended = False
    self.winner = None
    self.turns = 1

    # For first player
    player_1_choice = str(input("1st player: Do you want to be a human or a computer? (0 for human, 1 for computer"))
    symbol_1 = str(input("Choose a single letter symbol to be represented with."))
    while len(symbol_1)!=1:
      print ("The symbol must be one character, try again.")
      symbol_1 = str(input("Choose a symbol."))
    self.symbols[0] = symbol_1

    if player_1_choice == "0":
      identifier_1 = str(input("What would you like to be called?"))
      self.players[0] = Manual(identifier_1, self.symbols[0])
    elif player_1_choice == "1":
      identifier_1 = str(input("What would you like to be called?"))
      self.players[0] = Computer(identifier_1, self.symbols[0])
    
    # For player 2
    player_2_choice = str(input("2nd player: Do you want to be a human or a computer? (0 for human, 1 for computer"))
    symbol_2 = str(input("Choose a single letter symbol to be represented with."))
    while len(symbol_2)!=1:
      print ("The symbol must be one character, try again.")
      symbol_2 = str(input("Choose a symbol."))
    self.symbols[1] = symbol_2

    if player_2_choice == "0":
      identifier_2 = str(input("What would you like to be called?"))
      self.players[1] = Manual(identifier_2, self.symbols[1])
    elif player_2_choice == "1":
      identifier_2 = str(input("What would you like to be called?"))
      self.players[1] = Computer(identifier_2, self.symbols[1])

    print ("We have two players: {}({}) and {}({})".format(self.players[0].identifier, self.symbols[0], self.players[1].identifier, self.symbols[1]))
    self.rotation = self.players[0]

    self.state = []
    # Based on the size of the columns and rows, create a board.
    for row in range(self.ROWS):
      self.state.append([]) # Creates the rows
      for col in range(self.COLS):
        self.state[row].append(' ') # Adds in empty string for all the parts.

  def newGame(self):
    """
      So, users can easily play a new game while maintaing the scores from past games.
    """
    # Resets the values, except for the player attributes
    self.turns = 1
    self.ended = False
    self.winner = None
    self.rotation = self.players[0]
    self.state = []
    for i in range(self.ROWS):
      self.state.append([])
      for j in range(self.COLS):
        self.state[i].append(' ')

  def changerotation(self):
    # If player 1 made a move, next turn will be of player 2 and vice versa.
    # To denote this, we update the rotation attribute.
    if self.rotation == self.players[0]:
      self.rotation = self.players[1]
    else:
      self.rotation = self.players[0]
    # Increment the total number of turns in the game by 1.
    self.turns += 1

  def uiState(self):
    print("This is round - " + str(self.turns))
    print ("\t ----------------------------")
    for i in range(self.ROWS-1, -1, -1):
      print("\t", end="")
      for j in range(self.COLS):
        print("| " + str(self.state[i][j]), end=" ")
      print("|")
    print("\t ----------------------------")
    print("\t  1   2   3   4   5   6   7 ")

    # Identify the winner if the game was won, draw otherwise
    if self.ended:
      print("Game Has Ended!")
      # If we find a winner, print out their name.
      if self.winner != None:
        print(str(self.winner.identifier) + " is the winner")
      else:
        print("Oh no! There was a draw")
      
  def makeMove(self):
    active = self.rotation
    # Depending on the size of the grid, we will know when we've exhausted all moves.
    # Hence, the game draws.
    if self.turns > (self.ROWS * self.COLS):
      self.ended = True
      return
    
    # The move from the active player
    move = active.move(self.state)

    # Each player has choice to make that is equal to the number of columns
    for i in range(self.ROWS):
      # Inserts the active player's symbol, where the slot in the column is empty.
      if self.state[i][move] == ' ':
        self.state[i][move] = active.symbol
        # Change the turn
        # Check if someone won the game
        # Give out the state so the other player can make a move
        self.changerotation()
        self.foursExists()
        self.uiState()
        return
    # If the column is full, a piece cannot be placed there
    print ("No moves can be made in this column.")
    return

  # This function checks if there is a winner of the game.
  def foursExists(self):
    """
    As we saw from the makeMove method, everytime a move is made this methods is called.
    The goal is to check after every move, if there is a winner to the game
    And, if there if the game ends as represented by the "ended" variable.
    There are four ways the dots can be connected:
    * Vertically
    * Horizontally
    * Across in the +ve slope
    * Across in the -ve slope
    """
    for i in range(self.ROWS):
      for j in range(self.COLS):
        if self.state[i][j] != " ":
          # Checks for vertical fours.
          # If found, end the game.
          if self.verticalFours(i, j):
            self.ended = True
            return

          # Checks for horizontal fours.
          # If found, end the game.
          if self.horizontalFours(i, j):
            self.ended = True
            return

          # Checks for diagonal fours.
          # If found, end the game.
          # acrossFour returns the slope as well.
          diagFours, s = self.acrossFours(i, j)
          if diagFours:
            print (s)
            self.ended = True
            return
          

  def verticalFours(self, row, column):
    """
      Checks if there are any four vertical connections in a row.
      This is the check of the winning condition
      If there are four contiguous connection, the active player wins.
    """
    fourConnects = False
    contiguous = 0

    for i in range(row, self.ROWS):
      # Checks if there are contiguous pieces in the columns.
      if self.state[i][column].lower() == self.state[row][column].lower():
        contiguous += 1
      else:
        break

    # If there are four or more contiguous pieces, the player wins
    if contiguous >= 4:
      fourConnects = True
      if self.players[0].symbol.lower() == self.state[row][column].lower():
        self.winner = self.players[0]
      else:
        self.winner = self.players[1]
    return fourConnects

  def horizontalFours(self, row, column):
    """
      Checks if there are any four horizontol connections in a row.
      This is the check of the winning condition
      If there are four contiguous connection, the active player wins.
    """
    fourConnects = False
    contiguous = 0

    for i in range(column, self.COLS):
      # Checks if there are contiguous pieces in the columns.
      if self.state[row][i].lower() == self.state[row][column].lower():
        contiguous += 1
      else:
        break

    # If there are four or more contiguous pieces, the player wins
    if contiguous >= 4:
      fourConnects = True
      if self.players[0].symbol.lower() == self.state[row][column].lower():
        self.winner = self.players[0]
      else:
        self.winner = self.players[1]
    return fourConnects

  def acrossFours(self, row, column):
    """
      Across is a bit tricky, since it's not enough to just check the vertical and horizontal direction
      For that reason, we need to 
    """
    fourConnects = False
    c = 0
    s = None # For the slope

    contiguous = 0 # How many connects are made
    col = column
    for ro in range(row, self.ROWS):
        if col > self.ROWS:
            break
        # In case we find same pieces in positive slope direction /.
        elif self.state[ro][col].lower() == self.state[row][column].lower():
            contiguous += 1
        else:
            break
        col += 1

    # If we find 4 or more contiguous pieces, the player wins. 
    if contiguous >= 4:
        c += 1
        s = '+'
        # Check which player won.
        if self.players[0].symbol.lower() == self.state[row][column].lower():
            self.winner = self.players[0]
        else:
            self.winner = self.players[1]

    contiguous = 0
    col = column
    # We reverse the direction while checking for the negative slope \.
    for i in range(row, -1, -1):
        if col > self.ROWS:
            break
        elif self.state[i][col].lower() == self.state[row][column].lower():
            contiguous += 1
        else:
            break
        col += 1

    if contiguous >= 4:
        c += 1
        s = '-'
        # Check which player won.
        if self.players[0].symbol.lower() == self.state[row][column].lower():
            self.winner = self.players[0]
        else:
            self.winner = self.players[1]

    # c is only incremented when 4 pieces are connected diagonally.
    # If 'c' is more than 1, that means that we have found a winner.
    # If it is 2, that means that the connect is both positive and negative in slope.\/
    if c > 0:
        fourConnects = True
    if c == 2:
        s = '+-'
    # Return the slope and the result.
    return fourConnects, s
    
  def fours(self):
    """ 
      This is an optional step which only has visual relevance.
      It calls the notifyPlayer function, to highlight the four pieces that caused the player to win
    """ 
    for i in range(self.ROWS):
      for j in range(self.COLS):
        if self.state[i][j] != ' ':
          # "v" if vertical win
          if self.verticalFours(i, j):
            self.notifyPlayer(i, j, 'v')
          #"h" if horizontal win
          if self.horizontalFours(i, j):
            self.notifyPlayer(i, j, 'h')
          
          across_4, s = self.acrossFours(i, j)
          #"d" if diagonal win
          if across_4:
            self.notifyPlayer(i, j, 'd', s)
    
  def notifyPlayer(self, row, column, orien, s=None):
    """
      Helps see where the player won the game.
    """
    if orien == 'v':
      for i in range(4):
        self.state[row+i][column] = "*"
    
    elif orien == 'h':
      for i in range(4):
        self.state[row][column+i] = "*"
    
    elif orien == 'd':
      if s == '+' or s == '+-':
        for i in range(4):
          self.state[row+i][column+i] = "*"
    
      elif s == '-' or s == '+-':
        for i in range(4):
          self.state[row-i][column+i] = "*"
    
    else:
      print("Drawed.")
    


### Minimax Algorithm
Finally, we need to implement the Minimax Algorithm, which is called upon by the Computer class every time it needs to generate a new best move. The mechanisms with which it functions are neatly described in the code itself. But, the goal is to generate the best possible move for the computer given the current configuration of the board.

In [13]:
class Minimax(object):
  """
    This is the implementation of Minimax algorithm for Connect4.

  """
  st = None
  symbols = ["x", "o"]
    
  def __init__(self, st):
    # Creates a copy of the current state of the board
    self.st = [i[:] for i in st]
          
  def vertChk(self, row, col, state, conts):
  # Returns 1 if vertical four in a row is found.
    cont = 0

    for i in range(row, rows):
      # If we find a vertical four in a row 
      if state[i][col].lower() == state[row][col].lower():
          cont += 1
      else:
          break
    # If it finds contiguous possibilities greater than or equal to the conts value passed
    # We can return 1
    if cont >= conts:
      return 1
    else:
      return 0
  
  def horzChk(self, row, col, st, conts):
    """
      It's the same as vertical check.
    """
    cont = 0
    for j in range(col, 7):
      if st[row][j].lower() == st[row][col].lower():
        cont += 1
      else:
        break

    if cont >= conts:
      return 1
    else:
      return 0
  
  def acrossChk(self, row, col, st, conts):
    """
      Checks for diagonals
      Unlike vertical and horizontal one, we need to check for positive and negative slope.
    """
    c = 0
    cont = 0
    j = col
    for i in range(row, rows):
      if j > rows:
          break
      # If it finds contiguous values, add 1.
      elif st[i][j].lower() == st[row][col].lower():
          cont += 1
      else:
          break
      j += 1
    
    # If it finds contiguous values, add one to count 'c'.
    if cont >= conts:
      c += 1

    cont = 0
    j = col
    # Check for negative slop
    for i in range(row, -1, -1):
      if j > 6:
          break
      elif st[i][j].lower() == st[row][col].lower():
          cont += 1
      else:
          break
      j += 1

    if cont >= conts:
      c += 1

    # Return the final count.
    return c

  
  def nextMoveAI(self, depth, st, active_player):
    # Chooses the active and opposite player
    if active_player == self.symbols[0]:
      x_player = self.symbols[1]
    else:
      x_player = self.symbols[0]
    
    # Each of the possible moves will have an alpha value.
    # We store them in the dictionary.
    valid_moves = {}
    for col in range(cols):
        # if column i is a legal move...
      if self.movePossible(col, st):
          # make the move in column 'col' for active_player
        # We get a new state of the board now.
        # Until we exhaust the given depth, we search for the moves.
        temp = self.makeMove(st, col, active_player)
        valid_moves[col] = -self.search(depth-1, temp, x_player)
    
    # Our goal is the maximize alpha, so set it at the lowest possible value.
    highest_alpha = -999999999
    # Initialize the best move as None
    best_move = None
    # Randomly take from the valid moves
    moves = valid_moves.items()
    random.shuffle(list(moves))
    
    # From the list of valid moves, we take the one with the highest alpha value.
    for move, alpha in moves:
      # If aplha for the next move is higher than the existing highest alpha.
      if alpha >= highest_alpha:
        # Replace the highest_aplha with the new value
        highest_alpha = alpha
        # We now have a new best move
        best_move = move
    # After the loop ends, we should have the move with the largest alpha value
    # Hence, we return it as the best move.
    print (best_move, highest_alpha)
    return best_move, highest_alpha
        
  def search(self, depth, st, active_player):
    """ 
      This is the search function that recursively goes until we exhaust the depth limits.
      Returns the max alpha value
    """
    
    valid_moves = [] # Valid moves for a given state of the board
    for i in range(7):
      # If it is a legal move, then make the move
      if self.movePossible(i, st):
        temp = self.makeMove(st, i, active_player)
        # Add it to the valid_moves list
        valid_moves.append(temp)
    
    # Check if the depth is zero, or if this is a terminal node
    # We also check if any of the player has already gotten 4 connections.
    # In that case, isOver would be true and we return the value.
    if depth == 0 or len(valid_moves) == 0 or self.isOver(st):
      # Based on the heuristic, assigns the value.
      return self.value(st, active_player)
    
    # Checks for the active and opposite player
    if active_player == self.symbols[0]:
      x_player = self.symbols[1]
    else:
      x_player = self.symbols[0]

    # Set the alpha to the minimum value
    alpha = -9999999999
    
    for move in valid_moves:
      if move == None:
        print ("S")
        # Takes the max from the existing value and the one found from the search.
      alpha = max(alpha, -self.search(depth-1, move, x_player))
    return alpha


  def value(self, st, symbol):
    """
      This is a heuristic we will use for minimax
      Inspired byt: https://www.researchgate.net/publication/331552609_Research_on_Different_Heuristics_for_Minimax_Algorithm_Insight_from_Connect-4_Game
    """
    if symbol == self.symbols[0]:
      sym_opponent = self.symbols[1]
    else:
      sym_opponent = self.symbols[0]
    
    # Gather all the possible situations with two, three and four continuous pieces.
    ac_4 = self.findContPieces(st, symbol, 4) #Weighted the highest sicne it is a high chance of win
    ac_3 = self.findContPieces(st, symbol, 3) # If three in a row move is found
    ac_2 = self.findContPieces(st, symbol, 2) # If two in a row is found
    
    # For the opponent
    opp_fours = self.findContPieces(st, sym_opponent, 4)
    
    # If opponent has four contiguous pieces, return a large negative value
    # Opponent's win, is our loss
    if opp_fours > 0:
      return -100000
    
    else:
      # Return the heuristic value
      return ac_4*100000 + ac_3*100 + ac_2


#------------------------------------Helper methods ----------------------------_#

  def findContPieces(self, st, symbol, conts):
    count = 0
    # Find contiguous values with the length 'conts' starting at 'i','j'.
    for i in range(6):
      for j in range(7):
          # For vertical pieces.
        if st[i][j].lower() == symbol.lower():
            # Look for the four in a row condition vertically.
          count += self.vertChk(i, j, st, conts)
          
          # For horizontal pieces
          count += self.horzChk(i, j, st, conts)
          
          # For diagonal pieces
          count += self.acrossChk(i, j, st, conts)

    # Returns the addition of all the counts.
    return count

  def movePossible(self, column, st):
    for i in range(6):
      if st[i][column] == ' ':
          # In a column, if we find any empty string, we know that the move is possible.
        return True
    # If no empty string is found, then the column cannot add any more pieces.
    return False
    
  def isOver(self, st):
    # If we find continous pieces that is greater than 4, the game is over as a player wins.
    if self.findContPieces(st, self.symbols[0], 4) >= 1:
        return True
    # If the opponent find the 4 connected pieces, the game is over as well.
    elif self.findContPieces(st, self.symbols[1], 4) >= 1:
        return True
    else:
        return False
        
    
  def makeMove(self, st, column, symbol):
    t = [i[:] for i in st]
    for i in range(rows):
      # If the column is empty, add in the new piece.
      if t[i][column] == ' ':
        t[i][column] = symbol
        # Return the new configuration
        return t

          

Finally, we come to the main execution of the game itself. This runs a loop which ends only when a user wants to stop playing. There are all the necessary configurations in the game itself.

In [17]:
def printStats(p1, p2, outcome):
  print("{0}: {1} wins, {2}: {3} wins, {4} ties".format(p1.identifier,
    outcome[0], p2.identifier, outcome[1], outcome[2]))

# Main function
def main():
  # Create a connect four game
  game = ConnectFour()
  # Print the board with the empty state
  game.uiState()
  # Create two players based on the user input.
  p1 = game.players[0]
  p2 = game.players[1]
  
  # The game always has three outcomes
  # either one of the player wins, or there is a draw
  # We keep tally of that
  track_outcomes = [0, 0, 0]
  
  # At the end, we ask if the user wants to leave the game
  # Until the user says yes, we continue the game
  quit_game = False

  while not quit_game:
    # As long as there is no winner, or a draw, we keep making new moves
    # Turn by turn
    while not game.ended:
      game.makeMove()
    
    # Check for fours
    # Print the state out
    game.fours()
    game.uiState()

    # If p1 wins, add count.
    if game.winner == p1:
      track_outcomes[0] += 1
    
    # If p2 wins, add count.
    elif game.winner == p2:
      track_outcomes[1] += 1

    # If neither win, add draw
    elif game.winner == None:
      track_outcomes[2] += 1
    
    # Print the statistics from the games.
    printStats(p1, p2, track_outcomes)
    
    #Ask the user to play again with the same name and attributes
    while True:
      play_again = str(input("Would you like to play again? "))
      # Creates a new game, if yes
      if play_again == "yes": 
        game.newGame()
        game.uiState()
        break
      # Closes the game, if no
      elif play_again == "no":
        print("Thanks for playing!")
        exit = True
        # Break out from the while loop
        break
      else:
        print("yes or no??"),
        
        
main()



1st player: Do you want to be a human or a computer? (0 for human, 1 for computer1
Choose a single letter symbol to be represented with.comp1
The symbol must be one character, try again.
Choose a symbol.x
What would you like to be called?comp1
2nd player: Do you want to be a human or a computer? (0 for human, 1 for computer1
Choose a single letter symbol to be represented with.o
What would you like to be called?comp2
We have two players: comp1(x) and comp2(o)
This is round - 1
	 ----------------------------
	|   |   |   |   |   |   |   |
	|   |   |   |   |   |   |   |
	|   |   |   |   |   |   |   |
	|   |   |   |   |   |   |   |
	|   |   |   |   |   |   |   |
	|   |   |   |   |   |   |   |
	 ----------------------------
	  1   2   3   4   5   6   7 
This it the turn for comp1.  The symbol for comp1 if x
6 -1
This is round - 2
	 ----------------------------
	|   |   |   |   |   |   |   |
	|   |   |   |   |   |   |   |
	|   |   |   |   |   |   |   |
	|   |   |   |   |   |   |   |
	|   | 

KeyboardInterrupt: ignored

In [ ]:
f1
